<div style="background-color: #04D7FD; padding: 20px; text-align: left;">
    <h1 style="color: #000000; font-size: 36px; margin: 0;">Data Processing for RAG with Data Prep Kit (Python)</h1>
    
</div>


## Before Running the notebook

Please complete [setting up python dev environment](./setup-python-dev-env.md)

## Overview

This notebook will process PDF documents as part of RAG pipeline

![](media/rag-overview-2.png)

This notebook will perform steps 1, 2, 3 and 4 in RAG pipeline.

Here are the processing steps:

- **pdf2parquet** : Extract text (in markdown format) from PDF and store them as parquet files
- **Exact Dedup**: Documents with exact content are filtered out
- **Chunk documents**: Split the PDFs into 'meaningful sections' (paragraphs, sentences ..etc)
- **Text encoder**: Convert chunks into vectors using embedding models

## Step-1: Configuration

In [1]:
from my_config import MY_CONFIG

## Step-2:  Data

We will use white papers  about LLMs.  

- [Granite Code Models](https://arxiv.org/abs/2405.04324)
- [Attention is all you need](https://arxiv.org/abs/1706.03762)

You can of course substite your own data below

### 2.1 - Download data

In [2]:
import os, sys
import shutil
from utils import download_file

shutil.rmtree(MY_CONFIG.INPUT_DATA_DIR, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.INPUT_DATA_DIR, exist_ok=True)
print ("✅ Cleared input directory")
 
download_file (url = 'https://arxiv.org/pdf/1706.03762', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'attention.pdf' ))
download_file (url = 'https://arxiv.org/pdf/2405.04324', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'granite.pdf' ))
download_file (url = 'https://arxiv.org/pdf/2405.04324', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'granite2.pdf' )) # duplicate


✅ Cleared input directory

input/attention.pdf (2.22 MB) downloaded successfully.

input/granite.pdf (1.27 MB) downloaded successfully.

input/granite2.pdf (1.27 MB) downloaded successfully.


### 2.2 - Set input/output path variables for the pipeline

In [3]:
import os, sys
import shutil

if not os.path.exists(MY_CONFIG.INPUT_DATA_DIR ):
    raise Exception (f"❌ Input folder MY_CONFIG.INPUT_DATA_DIR = '{MY_CONFIG.INPUT_DATA_DIR}' not found")

output_parquet_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '01_parquet_out')
output_exact_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '02_dedupe_out')
output_chunk_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '03_chunk_out')
output_embeddings_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '04_embeddings_out')

## clear output folder
shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_FOLDER, exist_ok=True)

print ("✅ Cleared output directory")

✅ Cleared output directory


## Step-3: pdf2parquet -  Convert data from PDF to Parquet

This step is reading the input folder containing all PDF files and ingest them in a parquet table using the [Docling package](https://github.com/DS4SD/docling).
The documents are converted into a JSON format which allows to easily chunk it in the later steps.



### 3.1 - Execute 

In [4]:
%%time 

from dpk_pdf2parquet.transform_python import Pdf2Parquet
from dpk_pdf2parquet.transform import pdf2parquet_contents_types

STAGE = 1 
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{MY_CONFIG.INPUT_DATA_DIR}' --> output='{output_parquet_dir}'\n", flush=True)

result = Pdf2Parquet(input_folder= MY_CONFIG.INPUT_DATA_DIR,
                    output_folder= output_parquet_dir,
                    data_files_to_use=['.pdf'],
                    pdf2parquet_contents_type=pdf2parquet_contents_types.MARKDOWN,   # markdown
                    #    pdf2parquet_contents_type=pdf2parquet_contents_types.JSON   # JSON
                    ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-1: Processing input='input' --> output='output/01_parquet_out'



18:15:27 INFO - pdf2parquet parameters are : {'batch_size': -1, 'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.MARKDOWN: 'text/markdown'>, 'do_table_structure': True, 'do_ocr': True, 'ocr_engine': <pdf2parquet_ocr_engine.EASYOCR: 'easyocr'>, 'bitmap_area_threshold': 0.05, 'pdf_backend': <pdf2parquet_pdf_backend.DLPARSE_V2: 'dlparse_v2'>, 'double_precision': 8}
18:15:27 INFO - pipeline id pipeline_id
18:15:27 INFO - code location None
18:15:27 INFO - data factory data_ is using local data access: input_folder - input output_folder - output/01_parquet_out
18:15:27 INFO - data factory data_ max_files -1, n_sample -1
18:15:27 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.pdf'], files to checkpoint ['.parquet']
18:15:27 INFO - orchestrator pdf2parquet started at 2025-01-17 18:15:27
18:15:27 INFO - Number of files is 3, source profile {'max_file_size': 2.112621307373047, 'min_file_size': 1.2146415710

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

ERR#: COULD NOT CONVERT TO RS THIS TABLE TO COMPUTE SPANS


18:15:44 INFO - Completed 1 files (33.33%) in 0.219 min
18:16:12 INFO - Completed 2 files (66.67%) in 0.685 min
18:16:40 INFO - Completed 3 files (100.0%) in 1.155 min
18:16:40 INFO - Done processing 3 files, waiting for flush() completion.
18:16:40 INFO - done flushing in 0.0 sec
18:16:40 INFO - Completed execution in 1.214 min, execution result 0


✅ Stage:1 completed successfully
CPU times: user 3min 10s, sys: 10.9 s, total: 3min 21s
Wall time: 1min 15s


### 3.2 -  Inspect Generated output

Here we should see one entry per input file processed

In [5]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_parquet_dir)

# print ("Output dimensions (rows x columns)= ", output_df.shape)

output_df.head(5)

## To display certain columns
#parquet_df[['column1', 'column2', 'column3']].head(5)

,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename
0,attention.pdf,"Provided proper attribution is provided, Googl...",15,6,147,178f709f-cd23-4bad-957a-5e8a88c9af22,2949302674760005271,pdf,f1f600333e46c5d7e23f5a110a903ee38aab0bf7047eca...,46040,2025-01-17T18:15:44.573338,13.146994,attention.pdf
1,granite2.pdf,## Granite Code Models: A Family of Open Found...,28,19,295,758f58b8-eaba-444a-b348-d45194a1c2e6,3127757990743433032,pdf,0534b6a29ab9cedf21c3f6cf756cf0252d19a8e9135a41...,127678,2025-01-17T18:16:40.700160,28.162055,granite2.pdf
2,granite.pdf,## Granite Code Models: A Family of Open Found...,28,19,295,c19d4b3e-c045-4823-8814-43da8808d68d,3127757990743433032,pdf,0534b6a29ab9cedf21c3f6cf756cf0252d19a8e9135a41...,127678,2025-01-17T18:16:12.497813,27.883452,granite.pdf


## Step-4: Eliminate Duplicate Documents

We have 2 duplicate documnets here : `granite.pdf` and `granite2.pdf`.

Note how the `hash` for these documents are same.

We are going to perform **de-dupe**

On the content of each document, a SHA256 hash is computed, followed by de-duplication of record having identical hashes.

[Dedupe transform documentation](https://github.com/IBM/data-prep-kit/blob/dev/transforms/universal/ededup/README.md)

### 4.1 - Execute 

In [6]:
%%time 

from dpk_ededup.transform_python import Ededup

STAGE = 2
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_parquet_dir}' --> output='{output_exact_dedupe_dir}'\n", flush=True)

result = Ededup(input_folder=output_parquet_dir,
    output_folder=output_exact_dedupe_dir,
    ededup_doc_column="contents",
    ededup_doc_id_column="document_id"
    ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-2: Processing input='output/01_parquet_out' --> output='output/02_dedupe_out'



18:16:40 INFO - exact dedup params are {'doc_column': 'contents', 'doc_id_column': 'document_id', 'use_snapshot': False, 'snapshot_directory': None}
18:16:40 INFO - pipeline id pipeline_id
18:16:40 INFO - code location None
18:16:40 INFO - data factory data_ is using local data access: input_folder - output/01_parquet_out output_folder - output/02_dedupe_out
18:16:40 INFO - data factory data_ max_files -1, n_sample -1
18:16:40 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
18:16:40 INFO - orchestrator ededup started at 2025-01-17 18:16:40
18:16:40 INFO - Number of files is 3, source profile {'max_file_size': 0.04436779022216797, 'min_file_size': 0.02082538604736328, 'total_file_size': 0.10954761505126953}
18:16:40 INFO - Starting from the beginning
18:16:40 INFO - Completed 1 files (33.33%) in 0.0 min
18:16:40 INFO - Completed 2 files (66.67%) in 0.0 min
18:16:40 INFO - Com

✅ Stage:2 completed successfully
CPU times: user 32.8 ms, sys: 3.05 ms, total: 35.9 ms
Wall time: 32.6 ms


### 4.2 - Inspect Generated output

We would see 2 documents: `attention.pdf`  and `granite.pdf`.  The duplicate `granite.pdf` has been filtered out!

In [7]:
from utils import read_parquet_files_as_df

input_df = read_parquet_files_as_df(output_parquet_dir)
output_df = read_parquet_files_as_df(output_exact_dedupe_dir)

# print ("Input data dimensions (rows x columns)= ", input_df.shape)
# print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input files before exact dedupe : {input_df.shape[0]:,}")
print (f"Output files after exact dedupe : {output_df.shape[0]:,}")
print ("Duplicate files removed :  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(min(3, output_df.shape[0]))

Input files before exact dedupe : 3
Output files after exact dedupe : 2
Duplicate files removed :   1


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,removed
1,granite.pdf,## Granite Code Models: A Family of Open Found...,28,19,295,c19d4b3e-c045-4823-8814-43da8808d68d,3127757990743433032,pdf,0534b6a29ab9cedf21c3f6cf756cf0252d19a8e9135a41...,127678,2025-01-17T18:16:12.497813,27.883452,granite.pdf,[]
0,attention.pdf,"Provided proper attribution is provided, Googl...",15,6,147,178f709f-cd23-4bad-957a-5e8a88c9af22,2949302674760005271,pdf,f1f600333e46c5d7e23f5a110a903ee38aab0bf7047eca...,46040,2025-01-17T18:15:44.573338,13.146994,attention.pdf,[]


##  Step-5: Doc chunks

Split the documents in chunks.

[Chunking transform documentation](https://github.com/IBM/data-prep-kit/blob/dev/transforms/language/doc_chunk/README.md)

**Experiment with chunking size to find the setting that works best for your documents**

### 5.1 - Execute 

In [8]:
%%time

from dpk_doc_chunk.transform_python import DocChunk

STAGE = 3
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_exact_dedupe_dir}' --> output='{output_chunk_dir}'\n", flush=True)

result = DocChunk(input_folder=output_exact_dedupe_dir,
        output_folder=output_chunk_dir,
        doc_chunk_chunking_type= "li_markdown",
        # doc_chunk_chunking_type= "dl_json",
        doc_chunk_chunk_size_tokens = 128,  # default 128
        doc_chunk_chunk_overlap_tokens=30   # default 30
        ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-3: Processing input='output/02_dedupe_out' --> output='output/03_chunk_out'



18:16:41 INFO - doc_chunk parameters are : {'chunking_type': 'li_markdown', 'content_column_name': 'contents', 'doc_id_column_name': 'document_id', 'output_chunk_column_name': 'contents', 'output_source_doc_id_column_name': 'source_document_id', 'output_jsonpath_column_name': 'doc_jsonpath', 'output_pageno_column_name': 'page_number', 'output_bbox_column_name': 'bbox', 'chunk_size_tokens': 128, 'chunk_overlap_tokens': 30, 'dl_min_chunk_len': None}
18:16:41 INFO - pipeline id pipeline_id
18:16:41 INFO - code location None
18:16:41 INFO - data factory data_ is using local data access: input_folder - output/02_dedupe_out output_folder - output/03_chunk_out
18:16:41 INFO - data factory data_ max_files -1, n_sample -1
18:16:41 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
18:16:41 INFO - orchestrator doc_chunk started at 2025-01-17 18:16:41
18:16:41 INFO - Number of files is 3,

✅ Stage:3 completed successfully
CPU times: user 890 ms, sys: 87.7 ms, total: 978 ms
Wall time: 980 ms


### 5.2 - Inspect Generated output

We would see documents are split into many chunks

In [9]:
from utils import read_parquet_files_as_df

input_df = read_parquet_files_as_df(output_exact_dedupe_dir)  ## for debug purposes
output_df = read_parquet_files_as_df(output_chunk_dir)

print (f"Files processed : {input_df.shape[0]:,}")
print (f"Chunks created : {output_df.shape[0]:,}")

# print ("Input data dimensions (rows x columns)= ", input_df.shape)
# print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Files processed : 2
Chunks created : 60


,filename,num_pages,num_tables,num_doc_elements,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,removed,source_document_id,contents,document_id
48,granite.pdf,28,19,295,3127757990743433032,pdf,0534b6a29ab9cedf21c3f6cf756cf0252d19a8e9135a41...,127678,2025-01-17T18:16:12.497813,27.883452,granite.pdf,[],c19d4b3e-c045-4823-8814-43da8808d68d,"## 6.1.5 RepoBench, CrossCodeEval: Repository-...",63337c6952e14044ce448bb0dc6a369181b7779cffcd92...
35,granite.pdf,28,19,295,3127757990743433032,pdf,0534b6a29ab9cedf21c3f6cf756cf0252d19a8e9135a41...,127678,2025-01-17T18:16:12.497813,27.883452,granite.pdf,[],c19d4b3e-c045-4823-8814-43da8808d68d,## 3 Model Architecture\n\nWe train a series o...,b0ad58f3ab8f7e69f2460a6713bf65396737cb179cc374...
22,attention.pdf,15,6,147,2949302674760005271,pdf,f1f600333e46c5d7e23f5a110a903ee38aab0bf7047eca...,46040,2025-01-17T18:15:44.573338,13.146994,attention.pdf,[],178f709f-cd23-4bad-957a-5e8a88c9af22,## 6.2 Model Variations\n\nTo evaluate the imp...,60de5803d0837ef01773367a79da7c3e47fe90bec09ecb...


## Step-6:   Calculate Embeddings for Chunks

we will calculate embeddings for each chunk using an open source embedding model

[Embeddings / Text Encoder documentation](https://github.com/IBM/data-prep-kit/blob/dev/transforms/language/text_encoder/README.md)

### 6.1 - Execute

In [10]:
%%time 

from dpk_text_encoder.transform_python import TextEncoder

STAGE  = 4
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_chunk_dir}' --> output='{output_embeddings_dir}'\n", flush=True)


result = TextEncoder(input_folder= output_chunk_dir, 
               output_folder= output_embeddings_dir, 
               text_encoder_model_name = MY_CONFIG.EMBEDDING_MODEL
               ).transform()
if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-4: Processing input='output/03_chunk_out' --> output='output/04_embeddings_out'



18:16:42 INFO - text_encoder parameters are : {'content_column_name': 'contents', 'output_embeddings_column_name': 'embeddings', 'model_name': 'sentence-transformers/all-MiniLM-L6-v2'}
18:16:42 INFO - pipeline id pipeline_id
18:16:42 INFO - code location None
18:16:42 INFO - data factory data_ is using local data access: input_folder - output/03_chunk_out output_folder - output/04_embeddings_out
18:16:42 INFO - data factory data_ max_files -1, n_sample -1
18:16:42 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
18:16:42 INFO - orchestrator text_encoder started at 2025-01-17 18:16:42
18:16:42 INFO - Number of files is 2, source profile {'max_file_size': 0.04669189453125, 'min_file_size': 0.02893352508544922, 'total_file_size': 0.07562541961669922}
18:16:44 INFO - Completed 1 files (50.0%) in 0.003 min
18:16:45 INFO - Completed 2 files (100.0%) in 0.006 min
18:16:45 INFO - Don

✅ Stage:4 completed successfully
CPU times: user 1.03 s, sys: 132 ms, total: 1.16 s
Wall time: 3.21 s


### 6.2 - Inspect Generated output

In [11]:
from utils import read_parquet_files_as_df

input_df = read_parquet_files_as_df(output_chunk_dir)
output_df = read_parquet_files_as_df(output_embeddings_dir)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (60, 15)
Output data dimensions (rows x columns)=  (60, 16)


,filename,num_pages,num_tables,num_doc_elements,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,removed,source_document_id,contents,document_id,embeddings
2,attention.pdf,15,6,147,2949302674760005271,pdf,f1f600333e46c5d7e23f5a110a903ee38aab0bf7047eca...,46040,2025-01-17T18:15:44.573338,13.146994,attention.pdf,[],178f709f-cd23-4bad-957a-5e8a88c9af22,## Abstract\n\nThe dominant sequence transduct...,590629323f9d88598a80846d1df6a83d0ad6ac53efe278...,"[-0.08771476, -0.12373961, 0.043168165, 0.0060..."
18,attention.pdf,15,6,147,2949302674760005271,pdf,f1f600333e46c5d7e23f5a110a903ee38aab0bf7047eca...,46040,2025-01-17T18:15:44.573338,13.146994,attention.pdf,[],178f709f-cd23-4bad-957a-5e8a88c9af22,## 5.3 Optimizer\n\nWe used the Adam optimizer...,47fc3dca18355f0f161c953a7ad213eaa8c33da0be6875...,"[-0.0124165565, -0.04576251, 0.037190527, -0.0..."
21,attention.pdf,15,6,147,2949302674760005271,pdf,f1f600333e46c5d7e23f5a110a903ee38aab0bf7047eca...,46040,2025-01-17T18:15:44.573338,13.146994,attention.pdf,[],178f709f-cd23-4bad-957a-5e8a88c9af22,## 6.1 Machine Translation\n\nOn the WMT 2014 ...,b7aa340533889effd73d129e0c14083277031c44becfa6...,"[-0.037983608, -0.067570895, -0.000437462, 0.0..."


## Step-7: Copy output to final output dir

In [12]:
import shutil

shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER_FINAL, ignore_errors=True)
shutil.copytree(src=output_embeddings_dir, dst=MY_CONFIG.OUTPUT_FOLDER_FINAL)

print (f"✅ Copied output from '{output_embeddings_dir}' --> '{MY_CONFIG.OUTPUT_FOLDER_FINAL}'")

✅ Copied output from 'output/04_embeddings_out' --> 'output/output_final'
